# BIG TATA

In [63]:
import os
from pyspark.sql import SparkSession
from pyspark.sql.functions import col , sum as sum_func
from IPython.display import display

In [64]:
import findspark
findspark.init()
findspark.find()

'C:\\Program Files\\Spark\\spark-3.5.1-bin-hadoop3'

In [65]:
# Create a SparkSession
spark = SparkSession.builder \
    .appName("My Spark App") \
    .getOrCreate()

In [66]:
input_df = spark.read.csv("spotify_songs.csv", header=True, inferSchema=True)


In [67]:
# 3. Display schema of DataFrame. 
input_df.printSchema()


root
 |-- track_id: string (nullable = true)
 |-- track_name: string (nullable = true)
 |-- track_artist: string (nullable = true)
 |-- track_popularity: integer (nullable = true)
 |-- track_album_id: string (nullable = true)
 |-- track_album_name: string (nullable = true)
 |-- track_album_release_date: string (nullable = true)
 |-- playlist_name: string (nullable = true)
 |-- playlist_id: string (nullable = true)
 |-- playlist_genre: string (nullable = true)
 |-- playlist_subgenre: string (nullable = true)
 |-- danceability: string (nullable = true)
 |-- energy: double (nullable = true)
 |-- key: double (nullable = true)
 |-- loudness: double (nullable = true)
 |-- mode: double (nullable = true)
 |-- speechiness: double (nullable = true)
 |-- acousticness: double (nullable = true)
 |-- instrumentalness: double (nullable = true)
 |-- liveness: double (nullable = true)
 |-- valence: double (nullable = true)
 |-- tempo: double (nullable = true)
 |-- duration_ms: double (nullable = true)


In [68]:
# clean data

# remove unnecessary columns (track_album_id , track_album_name , track_album_release_date , playlist_name , playlist_id , track_popularity)

cleaned_df = input_df.drop("track_album_id","track_album_name","track_album_release_date","playlist_name","playlist_id" ,"track_popularity" , "track_name" , "track_artist")

# print schema
cleaned_df.printSchema()

# delete all records with null values



# show records count

print(cleaned_df.count())



root
 |-- track_id: string (nullable = true)
 |-- playlist_genre: string (nullable = true)
 |-- playlist_subgenre: string (nullable = true)
 |-- danceability: string (nullable = true)
 |-- energy: double (nullable = true)
 |-- key: double (nullable = true)
 |-- loudness: double (nullable = true)
 |-- mode: double (nullable = true)
 |-- speechiness: double (nullable = true)
 |-- acousticness: double (nullable = true)
 |-- instrumentalness: double (nullable = true)
 |-- liveness: double (nullable = true)
 |-- valence: double (nullable = true)
 |-- tempo: double (nullable = true)
 |-- duration_ms: double (nullable = true)
 |-- language: string (nullable = true)
 |-- lyrics: string (nullable = true)

18454


In [69]:


# #show records with null values

# columns = cleaned_df.columns

# for column in columns:
#     print ("records with null values in column " + column)
#     cleaned_df.filter(cleaned_df[column].isNull()).show()



In [70]:

# remove records with null values

cleaned_df = cleaned_df.na.drop()

# remove records with lyrics = NA
cleaned_df = cleaned_df.filter(cleaned_df.lyrics != "NA")
features = ["danceability","energy","key","loudness","mode","speechiness","acousticness","instrumentalness","liveness","valence","tempo","duration_ms"]

# make sure that all this features have numeric values

for feature in features:
    cleaned_df = cleaned_df.filter(col(feature).cast("float").isNotNull() | col(feature).cast("int").isNotNull() )
    

# show records count
print(cleaned_df.count())


18184


In [71]:



# show genre with more than 100 songs
result = cleaned_df.groupBy("playlist_genre").count()
filtered_result = result.filter(col("count") > 100).select("playlist_genre")
display(filtered_result.toPandas())


# remove all records that doesn't belong to filtered_result

cleaned_df = cleaned_df.join(filtered_result, on="playlist_genre", how="left")
print(cleaned_df.count())




,playlist_genre
0,r&b
1,pop
2,edm
3,rap
4,rock
5,latin


18184


In [72]:


# show subgenre with more than 100 songs
result = cleaned_df.groupBy("playlist_subgenre").count()
filtered_result = result.filter(col("count") > 100).select("playlist_subgenre")
display(filtered_result.toPandas())

# remove all records that doesn't belong to filtered_result

cleaned_df = cleaned_df.join(filtered_result, on="playlist_subgenre", how="left")

print(cleaned_df.count())


#export to csv
pandas_df = cleaned_df.toPandas()
pandas_df.to_csv("cleaned_songs.csv", index=False , header=True)








,playlist_subgenre
0,electropop
1,latin pop
2,tropical
3,album rock
4,new jack swing
5,urban contemporary
6,big room
7,pop edm
8,permanent wave
9,trap


18184


## EDA

In [73]:
cleaned_df = spark.read.csv("cleaned_songs.csv", header=True, inferSchema=True)

print(cleaned_df.count())

18184


In [74]:
# features : danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms

# draw distribution of each audio feature

features = ["danceability","energy","key","loudness","mode","speechiness","acousticness","instrumentalness","liveness","valence","tempo","duration_ms"]

for feature in features:
    display(cleaned_df.select(feature).summary().toPandas())





,summary,danceability
0,count,18184
1,mean,0.6442043004839418
2,stddev,0.15012583733803528
3,min,0.116
4,25%,0.545
5,50%,0.66
6,75%,0.7559999999999999
7,max,0.979


,summary,energy
0,count,18184
1,mean,0.6928828090629123
2,stddev,0.17926713617877324
3,min,0.0167
4,25%,0.574
5,50%,0.715
6,75%,0.8340000000000001
7,max,1.0


,summary,key
0,count,18184
1,mean,5.342498900131984
2,stddev,3.6148583732533655
3,min,0.0
4,25%,2.0
5,50%,6.0
6,75%,9.0
7,max,11.0


,summary,loudness
0,count,18184
1,mean,-6.749788275406952
2,stddev,2.893395674596352
3,min,-34.283
4,25%,-8.218
5,50%,-6.221
6,75%,-4.717
7,max,1.275


,summary,mode
0,count,18184
1,mean,0.5822151341838979
2,stddev,0.49320791668275055
3,min,0.0
4,25%,0.0
5,50%,1.0
6,75%,1.0
7,max,1.0


...

,summary,instrumentalness
0,count,18184
1,mean,0.04337248021722392
2,stddev,0.1506256884108591
3,min,0.0
4,25%,0.0
5,50%,7.42E-6
6,75%,0.00135
7,max,0.987


,summary,liveness
0,count,18184
1,mean,0.18939507039155312
2,stddev,0.15340515322901194
3,min,0.00936
4,25%,0.0926
5,50%,0.128
6,75%,0.246
7,max,0.996


,summary,valence
0,count,18184
1,mean,0.5228574356577211
2,stddev,0.22761868320033324
3,min,1.0E-5
4,25%,0.34700000000000003
5,50%,0.523
6,75%,0.7020000000000001
7,max,0.991


,summary,tempo
0,count,18184
1,mean,120.79643263308402
2,stddev,27.629675318437013
3,min,37.114
4,25%,98.528
5,50%,120.027
6,75%,136.00799999999998
7,max,214.047


,summary,duration_ms
0,count,18184
1,mean,230549.0757259129
2,stddev,56735.896677169534
3,min,31893.0
4,25%,193594.0
5,50%,221467.0
6,75%,258027.0
7,max,517810.0
